# Plan
1. Build a basic UI for testing purposes

# Generate Dummy Data

In [1]:
import pandas as pd
import numpy as np

import tempfile
import pathlib

tmpdir = pathlib.Path(tempfile.gettempdir())
dummy_csv_path = tmpdir / "dummy_data.csv"

token_classes = ["Nouns", "Verbs", "Adjectives"]
training_steps_options = [100000, 200000, 400000]
model_sizes_options = ["Small", "Medium", "Large"]

data = {
    "model_size": [],
    "training_steps": [],
    "loss": [],
    "token_class": [],
}

# Generate dummy data
for size in model_sizes_options:
    for steps in training_steps_options:
        for token_class in token_classes:
            data["model_size"].append(size)
            data["training_steps"].append(steps)
            data["token_class"].append(token_class)
            # loss should be random but decrease with size and steps
            loss = (
                1
                - (model_sizes_options.index(size) + 1) / len(model_sizes_options)
                - (training_steps_options.index(steps) + 1)
                / len(training_steps_options)
            )
            noisy_loss = loss + np.random.normal(0, 0.1)
            data["loss"].append(noisy_loss)

# Create DataFrame
dummy_df = pd.DataFrame(data)

# Save DataFrame to a CSV file
dummy_df.to_csv(dummy_csv_path, index=False)

dummy_csv_path

PosixPath('/var/folders/5k/7nfpl0cs5999pzhndyybcn800000gn/T/dummy_data.csv')

# Build Visualization

In [19]:
import pandas as pd
import plotly.graph_objs as go
from ipywidgets import interact, Dropdown
import plotly.express as px

# Load your data
df = pd.read_csv(dummy_csv_path)  # replace with your actual path

# dumb hack to avoid the first call to update_figure rendering a duplicate chart
_first_call = True

# Function to create and update the figure
def update_figure(comparison_type, model_size, training_steps, token_class):
    if comparison_type == 'model_size':
        filtered_df = df[(df['training_steps'] == training_steps) & (df['token_class'] == token_class)]
        fig = px.line(filtered_df, x='model_size', y='loss', title='Loss by Model Size')
    else:
        filtered_df = df[(df['model_size'] == model_size) & (df['token_class'] == token_class)]
        fig = px.line(filtered_df, x='training_steps', y='loss', title='Loss by Training Steps')
    
    global _first_call
    if _first_call:
        _first_call = False
    else:
        fig.show()

# Interactive widgets
comparison_type = Dropdown(options=['model_size', 'training_steps'])
model_size = Dropdown(options=sorted(df['model_size'].unique()))
training_steps = Dropdown(options=sorted(df['training_steps'].unique()))
token_class = Dropdown(options=df['token_class'].unique())

# only render the chart after all the widgets have been rendered
_ = interact(update_figure, comparison_type=comparison_type, model_size=model_size, training_steps=training_steps, token_class=token_class, __manual=True)

interactive(children=(Dropdown(description='comparison_type', options=('model_size', 'training_steps'), value=…

# Remaining Tasks

1. Get all token positions
2. Get all model token predictions
3. Get all 